In [1]:
import os
os.getcwd()

'/Users/varundesai/Visual_Studio_Varun/Projects/Chicken-Disease-Classification/notebooks'

In [2]:
os.chdir("../")
os.getcwd()

'/Users/varundesai/Visual_Studio_Varun/Projects/Chicken-Disease-Classification'

In [3]:
import tensorflow as tf

In [4]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [6]:
from chickenDiseaseClassification.constants import *
from chickenDiseaseClassification.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_validation_config(self)->EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.h5"),
            training_data=Path("artifacts/data_ingestion/Chicken-fecal-images"),
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [8]:
from urllib.parse import urlparse

In [9]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.25
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss" : self.score[0], "accuracy" : self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [10]:
try:
    config = ConfigurationManager()
    evaluation = Evaluation(config.get_validation_config())
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

[2024-02-27 18:09:48,863: INFO: common: 31: yaml file: config/config.yaml is loaded successfully]
[2024-02-27 18:09:48,865: INFO: common: 31: yaml file: params.yaml is loaded successfully]
[2024-02-27 18:09:48,866: INFO: common: 58: Creating directory: artifacts]
Found 96 images belonging to 2 classes.
6/6 [==============================] - 9s 1s/step - loss: 9.4582 - accuracy: 0.5000
[2024-02-27 18:09:57,721: INFO: common: 78: json file: scores.json is saved successfully]
